In [ ]:
# Imports
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import shutil
import csv
import yaml
from scipy import stats
import math
import datetime as dt


In [ ]:
def prepareFolder(path):
    if os.path.exists(path+'/') == True:
        shutil.rmtree(path)
    if os.path.exists(path+'/') == False:
        os.makedirs(path+'/')


In [ ]:
COL_NAMES = [
    # total-cpu-usage
    'usr: CPU usage by user processes [%]',
    'sys: CPU usage by system processes [%]',
    'idl: Number of idle processes [#]',
    'wai: Number of waiting processes [#]',
    'hiq: Number of hard interrupts [#]',
    'siq: Number of soft interrupts [#]',
    # memory usage
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    # https://linuxtect.com/linux-dstat-command-monitor-linux-performance-and-resource-usage/
    # https://www.thegeekdiary.com/understanding-proc-meminfo-file-analyzing-memory-utilization-in-linux/
    # /proc/meminfo
    'used: Amount of used memory [Bytes]',  # / 1024 = KB
    'buff: Amount of buffered memory [Bytes]',  # / 1024 / 1024 = MB
    'cach: Amount of cached memory [Bytes]',
    'free: Amount of free memory [Bytes]',

    # filesystem
    # https://www.linuxquestions.org/questions/linux-general-1/proc-sys-fs-file-nr-57646/
    # /proc/sys/fs/file-nr
    'files: Number of allocated file handles [#]',
    'inodes: Number of used file handles [#]',

    # dsk/total
    'read: Amount of read bytes on disk [Bytes]',  # not 100% sure
    'writ: Amount of written bytes on disk [Bytes]',  # / 1024 = KB

    # dsk/total
    'reads: Number of read operations on disk [#]',
    'writs: Number of write operations on disk [#]',

    # net/eth0
    'recv: Amount of received bytes on eth0 [B]',  # /1024 = k
    'send: Amount of received bytes on eth0 [B]',

    # tcp sockets
    # http://karunsubramanian.com/network/what-is-syn_sent-socket-status/
    # /proc/net/tcp
    # /proc/net/tcp6
    'lis: Number of TCP connections "listening" [#]',
    'act: Number of TCP connections "established" (active) [#]',
    'syn: Number of TCP connections "syn_s", "syn_r" or "last_a" [#]',
    'tim: Number of TCP connections with status "waiting"[#]',
    'clo: Number of TCP connections with status "closing"  [#]',

    # sockets
    # https://community.hpe.com/t5/Networking/Regarding-meaning-for-quot-cat-proc-net-sockstat-quot-results/td-p/3922867#.Yvnld3VBxhE
    # /proc/net/sockstat
    'tot: Number of total sockets [#]',
    'tcp: Number of tcp sockets [#]',
    'udp: Number of udp sockets [#]',
    'raw: Number of raw sockets [#]',
    'frg: Number of frag sockets [#]',

    # system
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    'int: Number of interrupts [#]',
    'csw: Number of ontext switches [#]',

    # procs
    # /proc/stat
    'run: Number of processes with status "running" [#]',
    'blk: Number of processes with status "blocked" [#]',
    'new: Number of processes with status "new" [#]'
]


In [ ]:
from df_loader import main
print(os.getcwd())
dfs, names = main()
# sanity check
for (df, name) in zip(dfs, names):
    print(df['usr'].iloc[0], name)


In [ ]:
# 00 healthy
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[0]
name = names[0]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = df[column+'-avg-healthy'].iloc[:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=0)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=0, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')


In [ ]:
# 01 httpBackdoor
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[1]
name = names[1]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware httpBackdoor')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')


In [ ]:
# 02 backdoor
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[2]
name = names[2]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware backdoor')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')


In [ ]:
# 03 The Tick
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[3]
name = names[3]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware The Tick')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')


In [ ]:
# 04 BASHLITE
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[4]
name = names[4]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware BASHLITE')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')


In [ ]:
# 05 Ransomware-PoC
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[5]
name = names[5]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:270].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[270:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[130], color='C1',
                      linewidth=1.75, linestyle='dotted')  # triggering MTD
    ax2 = plt.axvline(df.index[270], color='C1',
                      linewidth=2, linestyle='dotted')  # finished MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware Ransomware-PoC')  # triggering malware
    ax2 = plt.axvline(df.index[130], color='C1', linewidth=2,
                      linestyle='dotted', label='triggering MTD1')  # triggering MTD
    ax2 = plt.axvline(df.index[270], color='C1', linewidth=2,
                      linestyle='dotted', label='finished MTD1')  # finished MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:270].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[270:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[130], color='C1',
                      linewidth=2, linestyle='dotted')  # triggering MTD
    ax3 = plt.axvline(df.index[270], color='C1',
                      linewidth=2, linestyle='dotted')  # finished MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:270].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[270:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')


In [ ]:
# 06 BEURK
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[6]
name = names[6]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='dashed')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware BEURK')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='dashed', label='triggering MTD3')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='dashed')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')


In [ ]:
# 07 bdvl
prepareFolder('plots/timeseries')
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[7]
name = names[7]

os.mkdir('plots/timeseries/'+name)
index = 0

# MTD trigger point
for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='dashed')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware bdvl')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='dashed', label='triggering MTD3')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    # ax4 = df[column].plot(color='C1', alpha=0, linewidth=0, label='') # leave dummy for x ticks
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='dashed')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    plt.cla()
plt.close('all')
